<a href="https://colab.research.google.com/github/JANGJINHEE98/AIB-Jeju-weather-api-service/blob/main/third_project_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 전처리, 모델링 작업은 colab에서 수행했습니다. 

## ✔️ 전처리

In [30]:
import pandas as pd
from pandas import Series, DataFrame
import sqlite3
import datetime

In [31]:
con = sqlite3.connect("/content/last_db.db")
cur = con.cursor()
weather_df = pd.read_sql("SELECT * FROM weather", con, index_col=None)
airport_df = pd.read_sql("SELECT * FROM airport", con, index_col=None)

In [32]:
weather_df.head()

,index,observatoryName,baseDate,averageTemperature,lowestTemperature,lowestTemperatureTime,highestTemperature,highestTemperatureTime,dailyRainfall,maximumWindSpeed,maximumWindSpeedTime,averageWindSpeed,maximumWindSpeedDirection
0,0,우도,20191231,2.6000,1.2000,2126,6.7000,7,0.0000,23.1000,414,12.2000,288.0000
1,1,우도,20191230,11.7000,6.4000,2337,14.1000,59,0.0000,19.7000,2321,8.8000,291.4000
2,2,우도,20191229,12.7000,8.5000,2,15.6000,1933,33.0000,12.1000,1738,5.5000,113.3000
3,3,우도,20191228,7.7000,6.1000,5,9.6000,1251,0.0000,10.5000,203,3.4000,321.9000
4,4,우도,20191227,6.4000,5.7000,320,7.3000,8,0.0000,20.5000,24,9.7000,295.5000


In [33]:
airport_df.head()

,index,airport,date,pcg,pct,tof
0,0,CJU,20191231,0,913,I
1,1,CJU,20191230,0,967,I
2,2,CJU,20191229,0,998,I
3,3,CJU,20191228,0,865,I
4,4,CJU,20191227,0,801,I


In [34]:
airport_df[airport_df['tof'] == 'D']

,index,airport,date,pcg,pct,tof


In [35]:
airport_df = airport_df.drop(['pcg', 'tof'], axis=1)

In [36]:
weather_df = weather_df.drop(['observatoryName', 'index'], axis=1)
airport_df = airport_df.drop(['airport', 'index'], axis=1)

In [37]:
weather_df.columns

Index(['baseDate', 'averageTemperature', 'lowestTemperature',
       'lowestTemperatureTime', 'highestTemperature', 'highestTemperatureTime',
       'dailyRainfall', 'maximumWindSpeed', 'maximumWindSpeedTime',
       'averageWindSpeed', 'maximumWindSpeedDirection'],
      dtype='object')

In [38]:
airport_df.columns

Index(['date', 'pct'], dtype='object')

In [39]:
weather_df[['baseDate', 'averageTemperature', 'lowestTemperature', 'lowestTemperatureTime', 'highestTemperature', 'highestTemperatureTime', 'dailyRainfall', 'maximumWindSpeed', 'maximumWindSpeedTime','averageWindSpeed', 'maximumWindSpeedDirection']] = weather_df[['baseDate', 'averageTemperature', 'lowestTemperature', 'lowestTemperatureTime', 'highestTemperature', 'highestTemperatureTime', 'dailyRainfall', 'maximumWindSpeed', 'maximumWindSpeedTime', 'averageWindSpeed', 'maximumWindSpeedDirection']].apply(pd.to_numeric)

In [40]:
airport_df[airport_df['date'].isnull()]

,date,pct
718,None,None


In [41]:
airport_df.loc[717]

date    20180113
pct          353
Name: 717, dtype: object

In [42]:
airport_df.loc[719]

date    20180111
pct          173
Name: 719, dtype: object

In [43]:
# 빠진 부분 메꿔넣기 ('20180112'가 들어가야 함)

In [44]:
airport_df.loc[718, 'date'] = '20180112'

In [45]:
airport_df.loc[718]

date    20180112
pct         None
Name: 718, dtype: object

In [46]:
airport_df[['date', 'pct']] = airport_df[['date', 'pct']].apply(pd.to_numeric)

In [47]:
weather_df.rename(columns = {'baseDate' : 'date'}, inplace = True)

In [48]:
airport_df['day'] = pd.to_datetime(airport_df['date'], format="%Y%m%d")

In [49]:
airport_df['day'] = airport_df['day'].dt.day_name()

In [50]:
df = pd.merge(weather_df, airport_df, on='date')

In [51]:
df = df.dropna(axis=0)

In [52]:
df.shape

(870, 13)

In [53]:
df['datetime'] = pd.to_datetime(df['date'], format="%Y%m%d")

In [54]:
df['M'] = df['datetime'].dt.month

In [55]:
df['D'] = df['datetime'].dt.day 

In [56]:
df.head()

,date,averageTemperature,lowestTemperature,lowestTemperatureTime,highestTemperature,highestTemperatureTime,dailyRainfall,maximumWindSpeed,maximumWindSpeedTime,averageWindSpeed,maximumWindSpeedDirection,pct,day,datetime,M,D
0,20191231,2.6,1.2,2126.0,6.7,7.0,0.0,23.1,414.0,12.2,288.0,913.0,Tuesday,2019-12-31,12,31
1,20191230,11.7,6.4,2337.0,14.1,59.0,0.0,19.7,2321.0,8.8,291.4,967.0,Monday,2019-12-30,12,30
2,20191229,12.7,8.5,2.0,15.6,1933.0,33.0,12.1,1738.0,5.5,113.3,998.0,Sunday,2019-12-29,12,29
3,20191228,7.7,6.1,5.0,9.6,1251.0,0.0,10.5,203.0,3.4,321.9,865.0,Saturday,2019-12-28,12,28
4,20191227,6.4,5.7,320.0,7.3,8.0,0.0,20.5,24.0,9.7,295.5,801.0,Friday,2019-12-27,12,27


In [57]:
df['MD'] = df['date'].astype(str).str[4:]

In [58]:
df['pct'] = df['pct']*9

In [59]:
df

,date,averageTemperature,lowestTemperature,lowestTemperatureTime,highestTemperature,highestTemperatureTime,dailyRainfall,maximumWindSpeed,maximumWindSpeedTime,averageWindSpeed,maximumWindSpeedDirection,pct,day,datetime,M,D,MD
0,20191231,2.6,1.2,2126.0,6.7,7.0,0.0,23.1,414.0,12.2,288.0,8217.0,Tuesday,2019-12-31,12,31,1231
1,20191230,11.7,6.4,2337.0,14.1,59.0,0.0,19.7,2321.0,8.8,291.4,8703.0,Monday,2019-12-30,12,30,1230
2,20191229,12.7,8.5,2.0,15.6,1933.0,33.0,12.1,1738.0,5.5,113.3,8982.0,Sunday,2019-12-29,12,29,1229
3,20191228,7.7,6.1,5.0,9.6,1251.0,0.0,10.5,203.0,3.4,321.9,7785.0,Saturday,2019-12-28,12,28,1228
4,20191227,6.4,5.7,320.0,7.3,8.0,0.0,20.5,24.0,9.7,295.5,7209.0,Friday,2019-12-27,12,27,1227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20170719,26.2,23.8,548.0,29.9,1358.0,0.0,4.9,1718.0,0.9,250.3,2547.0,Wednesday,2017-07-19,7,19,0719
896,20170718,25.5,23.5,648.0,27.3,852.0,55.5,7.3,1611.0,1.6,196.9,1791.0,Tuesday,2017-07-18,7,18,0718
897,20170717,27.6,26.8,523.0,29.3,1348.0,0.0,10.6,119.0,3.6,213.8,1620.0,Monday,2017-07-17,7,17,0717
898,20170716,28.0,26.6,1948.0,29.6,1226.0,0.0,11.6,316.0,4.7,219.4,1683.0,Sunday,2017-07-16,7,16,0716


In [60]:
# con = sqlite3.connect("/content/last_db.db")
# cur = con.cursor()
# df.to_sql('df_for_dashboard', con, if_exists='replace')

In [61]:
# pd.read_sql("SELECT * FROM df_for_dashboard", con, index_col=None)

,index,date,averageTemperature,lowestTemperature,lowestTemperatureTime,highestTemperature,highestTemperatureTime,dailyRainfall,maximumWindSpeed,maximumWindSpeedTime,averageWindSpeed,maximumWindSpeedDirection,pct,day,datetime,M,D,MD
0,0,20191231,2.6,1.2,2126.0,6.7,7.0,0.0,23.1,414.0,12.2,288.0,8217.0,Tuesday,2019-12-31 00:00:00,12,31,1231
1,1,20191230,11.7,6.4,2337.0,14.1,59.0,0.0,19.7,2321.0,8.8,291.4,8703.0,Monday,2019-12-30 00:00:00,12,30,1230
2,2,20191229,12.7,8.5,2.0,15.6,1933.0,33.0,12.1,1738.0,5.5,113.3,8982.0,Sunday,2019-12-29 00:00:00,12,29,1229
3,3,20191228,7.7,6.1,5.0,9.6,1251.0,0.0,10.5,203.0,3.4,321.9,7785.0,Saturday,2019-12-28 00:00:00,12,28,1228
4,4,20191227,6.4,5.7,320.0,7.3,8.0,0.0,20.5,24.0,9.7,295.5,7209.0,Friday,2019-12-27 00:00:00,12,27,1227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,895,20170719,26.2,23.8,548.0,29.9,1358.0,0.0,4.9,1718.0,0.9,250.3,2547.0,Wednesday,2017-07-19 00:00:00,7,19,0719
866,896,20170718,25.5,23.5,648.0,27.3,852.0,55.5,7.3,1611.0,1.6,196.9,1791.0,Tuesday,2017-07-18 00:00:00,7,18,0718
867,897,20170717,27.6,26.8,523.0,29.3,1348.0,0.0,10.6,119.0,3.6,213.8,1620.0,Monday,2017-07-17 00:00:00,7,17,0717
868,898,20170716,28.0,26.6,1948.0,29.6,1226.0,0.0,11.6,316.0,4.7,219.4,1683.0,Sunday,2017-07-16 00:00:00,7,16,0716


In [62]:
con.close()

In [ ]:
df_get_weather = df.drop(['date', 'pct', 'M', 'D'], axis=1)

In [ ]:
weather_predict = df_get_weather.groupby('MD').mean() # 데이터 베이스에 저장해야 할까? 

In [ ]:
# con = sqlite3.connect("/content/third_project_db.db")
# cur = con.cursor()
# weather_predict.to_sql('weather_predict', con, if_exists='replace')

In [ ]:
# 데이터가 잘 들어갔는지 확인 
# pd.read_sql("SELECT * FROM weather_predict", con, index_col=None)

In [ ]:
# con.close()

In [ ]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870 entries, 0 to 899
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       870 non-null    int64         
 1   averageTemperature         870 non-null    float64       
 2   lowestTemperature          870 non-null    float64       
 3   lowestTemperatureTime      870 non-null    float64       
 4   highestTemperature         870 non-null    float64       
 5   highestTemperatureTime     870 non-null    float64       
 6   dailyRainfall              870 non-null    float64       
 7   maximumWindSpeed           870 non-null    float64       
 8   maximumWindSpeedTime       870 non-null    float64       
 9   averageWindSpeed           870 non-null    float64       
 10  maximumWindSpeedDirection  870 non-null    float64       
 11  pct                        870 non-null    float64       
 12  day     

In [ ]:
df = df.drop(['datetime', 'date'], axis=1)

In [ ]:
df['MD'] = df['MD'].apply(pd.to_numeric)

In [ ]:
pip install category_encoders

     |████████████████████████████████| 80 kB 8.4 MB/s 


## ✔️ 모델링

In [ ]:
#모델 학습 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# ecoding 
encoder = OneHotEncoder(use_cat_names=True)
encoder.fit(X = df)
encoded_df = encoder.transform(df)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
y = encoded_df['pct']
x = encoded_df.drop('pct', axis=1)

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=1004)

In [ ]:
x

,averageTemperature,lowestTemperature,lowestTemperatureTime,highestTemperature,highestTemperatureTime,dailyRainfall,maximumWindSpeed,maximumWindSpeedTime,averageWindSpeed,maximumWindSpeedDirection,day_Tuesday,day_Monday,day_Sunday,day_Saturday,day_Friday,day_Thursday,day_Wednesday,M,D,MD
0,2.6,1.2,2126.0,6.7,7.0,0.0,23.1,414.0,12.2,288.0,1,0,0,0,0,0,0,12,31,1231
1,11.7,6.4,2337.0,14.1,59.0,0.0,19.7,2321.0,8.8,291.4,0,1,0,0,0,0,0,12,30,1230
2,12.7,8.5,2.0,15.6,1933.0,33.0,12.1,1738.0,5.5,113.3,0,0,1,0,0,0,0,12,29,1229
3,7.7,6.1,5.0,9.6,1251.0,0.0,10.5,203.0,3.4,321.9,0,0,0,1,0,0,0,12,28,1228
4,6.4,5.7,320.0,7.3,8.0,0.0,20.5,24.0,9.7,295.5,0,0,0,0,1,0,0,12,27,1227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,26.2,23.8,548.0,29.9,1358.0,0.0,4.9,1718.0,0.9,250.3,0,0,0,0,0,0,1,7,19,719
896,25.5,23.5,648.0,27.3,852.0,55.5,7.3,1611.0,1.6,196.9,1,0,0,0,0,0,0,7,18,718
897,27.6,26.8,523.0,29.3,1348.0,0.0,10.6,119.0,3.6,213.8,0,1,0,0,0,0,0,7,17,717
898,28.0,26.6,1948.0,29.6,1226.0,0.0,11.6,316.0,4.7,219.4,0,0,1,0,0,0,0,7,16,716


In [ ]:
x.shape

(870, 20)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870 entries, 0 to 899
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   averageTemperature         870 non-null    float64
 1   lowestTemperature          870 non-null    float64
 2   lowestTemperatureTime      870 non-null    float64
 3   highestTemperature         870 non-null    float64
 4   highestTemperatureTime     870 non-null    float64
 5   dailyRainfall              870 non-null    float64
 6   maximumWindSpeed           870 non-null    float64
 7   maximumWindSpeedTime       870 non-null    float64
 8   averageWindSpeed           870 non-null    float64
 9   maximumWindSpeedDirection  870 non-null    float64
 10  day_Tuesday                870 non-null    int64  
 11  day_Monday                 870 non-null    int64  
 12  day_Sunday                 870 non-null    int64  
 13  day_Saturday               870 non-null    int64  

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
pred = model.predict(X_test)
r2 = r2_score(y_test, pred)

In [ ]:
r2

-60.0782078115402

In [ ]:
# 모델 저장 
import pickle
from sklearn.externals import joblib
saved_model = pickle.dumps(model)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(saved_model)

b'\x80\x03csklearn.linear_model._base\nLinearRegression\nq\x00)\x81q\x01}q\x02(X\r\x00\x00\x00fit_interceptq\x03\x88X\t\x00\x00\x00normalizeq\x04\x89X\x06\x00\x00\x00copy_Xq\x05\x88X\x06\x00\x00\x00n_jobsq\x06NX\x05\x00\x00\x00coef_q\x07cnumpy.core.multiarray\n_reconstruct\nq\x08cnumpy\nndarray\nq\tK\x00\x85q\nC\x01bq\x0b\x87q\x0cRq\r(K\x01K\x14\x85q\x0ecnumpy\ndtype\nq\x0fX\x02\x00\x00\x00f8q\x10\x89\x88\x87q\x11Rq\x12(K\x03X\x01\x00\x00\x00<q\x13NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x14b\x89C\xa0\x1f\x00!\x8c\xb1\xa8-@]Fo\xeaE\x85\xd9?\xd2N4\xc6g\x87\x89\xbf\xaf#h\xc0\xcd\xd1\xf1\xbf\xa4l\x9f#\x9f\xcf\x9e\xbf\xc2\x1e\xe4\x95\xe5u\xe4?=\xb7\xec"&_2\xc0\xb9\x03S\x13\x8f\xaf\x88\xbfoeO\x06"\x1aM@\xaa\xef\x9f\xd4\xd8\x94\xd2?\xe3a\x0cV\x9b\x98J\xc0\x13\xf6\xcf|["K\xc0MS%\x00\xbf\xe1^@J\xb2\xb2@\xa9xS@\xea4\xa4\xed\xa4\xefS\xc0\xba\x1e\x02\xfa\xe0{Q@`\xc3\xc7\xe3(\tU\xc0V\xcc0bk\x00\x85\xbf|!\xd7J\xc0]\xf0?@\xebI\xbd3\'e\xbfq\x15tq\x16bX\t\x00\x00\x00_residuesq\x17h\x08h\tK\x00\x85

In [ ]:
joblib.dump(model, 'pickled_model.pkl') 

['pickled_model.pkl']